In [22]:
# get data for certain network_id
network_id = 'santander-cycles'
network_json = requests.get("http://api.citybik.es/v2/networks/"+network_id).json()
network_json

{'network': {'id': 'santander-cycles',
  'name': 'Santander Cycles',
  'location': {'latitude': 51.51121389999999,
   'longitude': -0.1198244,
   'city': 'London',
   'country': 'GB'},
  'href': '/v2/networks/santander-cycles',
  'company': ['PBSC', 'Serco Group plc'],
  'system': 'Bixi',
  'stations': [{'id': '001420f03e8b4f08f1bcc9bdc0260651',
    'name': '300073 - Prince of Wales Drive, Battersea Park',
    'latitude': 51.47515398,
    'longitude': -0.159169801,
    'timestamp': '2025-04-10T03:05:34.593941Z',
    'free_bikes': 12,
    'empty_slots': 3,
    'extra': {'uid': 756,
     'name': 'Prince of Wales Drive, Battersea Park',
     'terminalName': '300073',
     'locked': False,
     'installed': True,
     'temporary': False,
     'installDate': '1390223040000',
     'removalDate': ''}},
   {'id': '004cab6485d7adb50b6a85cbe152399a',
    'name': '003447 - Gloucester Road (North), Kensington',
    'latitude': 51.49792478,
    'longitude': -0.183834706,
    'timestamp': '2025-04-1

'S'

,id,name,latitude,longitude,timestamp,free_bikes,empty_slots,extra.uid,extra.name,extra.terminalName,extra.locked,extra.installed,extra.temporary,extra.installDate,extra.removalDate
0,001420f03e8b4f08f1bcc9bdc0260651,"300073 - Prince of Wales Drive, Battersea Park",51.475154,-0.159170,2025-04-10T03:05:34.593941Z,12,3,756,"Prince of Wales Drive, Battersea Park",300073,False,True,False,1390223040000,
1,004cab6485d7adb50b6a85cbe152399a,"003447 - Gloucester Road (North), Kensington",51.497925,-0.183835,2025-04-10T03:05:34.298392Z,1,15,97,"Gloucester Road (North), Kensington",003447,False,True,False,1279097580000,
2,00531c4ce0f2e75ef2cebf19f1a6c1e3,"200213 - Durant Street, Bethnal Green",51.528682,-0.065550,2025-04-10T03:05:34.585717Z,21,0,702,"Durant Street, Bethnal Green",200213,False,True,False,1384345860000,
3,0191ad8ece6d587cff8369b58327aacb,"003488 - Charles II Street, West End",51.508446,-0.131961,2025-04-10T03:05:34.459778Z,0,24,226,"Charles II Street, West End",003488,False,True,False,1279616520000,
4,01995d3d3cff733f43d7809d397b3a7e,"001126 - North Audley Street, Mayfair",51.512912,-0.153645,2025-04-10T03:05:34.358658Z,1,14,180,"North Audley Street, Mayfair",001126,False,True,False,1279298220000,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
790,fefbd80de948379a493be17f28907a2c,"200071 - Hoxton Street, Hoxton",51.529590,-0.080100,2025-04-10T03:05:34.548658Z,11,16,588,"Hoxton Street, Hoxton",200071,False,True,False,1330937820000,
791,ff26abff5db37c1b3563bd0c430476b2,"003442 - Abingdon Villas, Kensington",51.497388,-0.197246,2025-04-10T03:05:34.289584Z,12,3,38,"Abingdon Villas, Kensington",003442,False,True,False,1278584880000,
792,ff29665797070bd57a6f6fab26ea5f1f,"002680 - Harrowby Street, Marylebone",51.517372,-0.164208,2025-04-10T03:05:34.492164Z,2,14,367,"Harrowby Street, Marylebone",002680,False,True,False,,
793,ffb92bc589f72ac1783e5696b0714e01,"003462 - Royal Avenue 2, Chelsea",51.490083,-0.162418,2025-04-10T03:05:34.480283Z,6,4,280,"Royal Avenue 2, Chelsea",003462,False,True,False,1279726440000,


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [19]:
# get foursquare data
https://api.foursquare.com/v3/places/search?ll=51.4769,-0.0005&radius=1000

SyntaxError: invalid syntax (2093899832.py, line 2)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating